In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [12]:
from datasets import load_dataset
data_files = {"train": "data.csv"}
dataset = load_dataset("scherrmann/financial_phrasebank_75agree_german")
df = pd.DataFrame(dataset['train'])

In [13]:
df.isnull().sum()

sentence    0
label       0
dtype: int64

In [ ]:

# sentiment_map = {'negative': 0, 'neutral': 1, 'positive': 2}
sentiment_map = {0: 'negative', 1: 'neutral', 2: 'positive'}
df['Sentiment'] = df['label'].map(sentiment_map)
df.rename(columns={'sentence': 'Sentence'}, inplace=True)

In [21]:
df['Sentiment'].value_counts()

Sentiment
neutral     1717
positive     710
negative     336
Name: count, dtype: int64

In [ ]:
train, eval = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Sentiment'])
train.shape, eval.shape

((4673, 2), (1169, 2))

In [ ]:
train.to_csv("data/train_subset.csv", index=False)
eval.to_csv("data/eval_subset.csv", index=False)